In [1]:
import requests
import pandas as pd

In [2]:
def get_top_movies_per_month(year_list):
        #yearlist = [2020,2019,2040]
        frames = []
        
        for year in year_list:
            for month in range(1,13):
                url = 'https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/{}/{}/{}/{}/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-domestic-box-office=On&show-genre=On'.format(year, year, month, month)
                html_page = requests.get(url)
                #frames.append(pd.read_html(html_page.text, index_col=0)[0])
                #frames.append(pd.read_html(html_page.text)[0])
                frames.append(pd.read_html(html_page.text, index_col=None)[0])
                
        return pd.concat(frames)
    
    
def no_char(col):
    # enter dataframe column to return string of the column back w/o '$' and ','.
    df1 = col
   
    df1 = df1.str.replace('$', '')
    df1 = df1.str.replace(',', '')
    
    return df1

In [3]:
# create a list of years to scrape
year_list = [2020, 2019, 2018, 2017, 2016]

#creates a dataframe of scraped data
df = get_top_movies_per_month(year_list)

In [4]:
#renames column and limits data to movies that made at least a dollar
df = df.rename(columns={"DomesticBox Office":"DomesticBox_Office"})
df = df[df.DomesticBox_Office != "$0" ]

#takes the '$' and ',' out of the string. Example: "$1,200,000" to "12000000"
df["DomesticBox_Office"] = no_char(df["DomesticBox_Office"])

#because there isn't any missing values in DomesticBox_office, we can immediatly change column to int
df['DomesticBox_Office'] = df['DomesticBox_Office'].astype(int)

#filling nans in 'Genre' column to "other" 
df['Genre'] = df['Genre'].fillna("Other")

#sort rows by DomesticBox_Office values
df = df.sort_values("DomesticBox_Office", ascending=False)

#reset index based of rank
df = df.reset_index()

#convert 'Released' column to Datetime to sparse out new columns 'Month' and 'year'
df['Released'] = pd.to_datetime(df['Released'])
df['Month'] = df['Released'].dt.month
df['Year'] = df['Released'].dt.year

#dropping unneccerary columns
df = df.drop("Released", axis=1)
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("index", axis=1)

In [5]:
#exports the cleaned table to a .csv file
#also makes for a nice save point for future work
df.to_csv('Cleaned_movie_data.csv')

In [6]:
df

,Title,Genre,DomesticBox_Office,Month,Year
0,Avengers: Endgame,Action,858373000,4,2019
1,Black Panther,Action,700059566,2,2018
2,Avengers: Infinity War,Action,678815482,4,2018
3,Star Wars Ep. VIII: The Last Jedi,Adventure,620181382,12,2017
4,Incredibles 2,Adventure,608581744,6,2018
...,...,...,...,...,...
2655,Mobile Homes,Drama,449,4,2018
2656,2:22,Thriller/Suspense,422,6,2017
2657,Des nouvelles de la planÃ¨te Mars,Comedy,310,7,2016
2658,Satanic,Horror,252,7,2016
